**Incomplete project**

In [27]:
import pandas as pd
import numpy as np

In [28]:
akas_url = 'https://datasets.imdbws.com/title.akas.tsv.gz'
basics_url = 'https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_url = 'https://datasets.imdbws.com/title.ratings.tsv.gz'

In [29]:
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

In [30]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


# Processing

In [31]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8939032 entries, 0 to 8939031
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 613.8+ MB


In [32]:
basics.duplicated().sum()

0

In [33]:
basics.isna().sum()

tconst             0
titleType          0
primaryTitle      11
originalTitle     11
isAdult            0
startYear          0
endYear            0
runtimeMinutes     0
genres            10
dtype: int64

In [34]:
# replacing all '\N' values with np.nan
basics = basics.replace({'\\N':np.nan})
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1192047
endYear           8848186
runtimeMinutes    6534018
genres             408409
dtype: int64

In [35]:
# basics = basics.dropna()
# basics.isna().sum()

In [36]:
# Eliminating movies that are null for 'runtimeMinutes' and 'genre'
basics = basics[['runtimeMinutes','genres']].dropna()
basics.isna().sum()

runtimeMinutes    0
genres            0
dtype: int64

In [37]:
# Excluding movies without the titleType movie 
basics['titleType'] = basics.loc[basics['titleType'] != 'Movie']

KeyError: 'titleType'